In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import numpy as np
import keras as k
import json
import PIL

In [59]:
def imagegen():
    f = open('/content/drive/My Drive/InfraredSolarModules/module_metadata.json',) 
    meta = json.load(f) 
    f.close() 
    img = []
    label = []
    for i in range(9980,10020):
        img.append(np.array(k.preprocessing.image.load_img('/content/drive/My Drive/InfraredSolarModules/' + meta[str(i)]['image_filepath'], target_size=(224, 224))))
        if meta[str(i)]['anomaly_class'] == 'No-Anomaly':
          label.append('No-Anomaly')
        else:
          label.append('Anomaly')  
    return np.array(img), np.array(label)
 
imgs, labels = imagegen()



In [60]:
print(len(labels))

40


In [9]:
type(imgs)

numpy.ndarray

In [42]:
imgs.shape

(10, 224, 224)

In [10]:
f = open('/content/drive/My Drive/InfraredSolarModules/module_metadata.json',) 
meta = json.load(f) 
f.close() 

In [61]:
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16 
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image

model = VGG16(weights="imagenet", include_top=False, classes=2)

def preprocess_image(imgs):
  preprocessed_images = []

  for image_item in imgs:
    image_item = image.img_to_array(image_item)
    image_item = np.expand_dims(image_item, axis=0)
    image_item = preprocess_input(image_item) 

preprocess_image(imgs)


features = model.predict(imgs)
print(features.shape)

(40, 7, 7, 512)


In [15]:
img.shape

(10, 224, 224, 1)

In [64]:
from sklearn import svm
from sklearn.model_selection import train_test_split
features = features.reshape(40,7*7*512)
svm_classifier = svm.SVC(kernel='linear', C=10)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)



In [65]:
print(X_train.shape)

print(X_train.shape)
print(y_train)

(30, 25088)
(30, 25088)
['Anomaly' 'Anomaly' 'Anomaly' 'Anomaly' 'Anomaly' 'No-Anomaly'
 'No-Anomaly' 'Anomaly' 'No-Anomaly' 'No-Anomaly' 'No-Anomaly' 'Anomaly'
 'Anomaly' 'Anomaly' 'Anomaly' 'Anomaly' 'Anomaly' 'Anomaly' 'No-Anomaly'
 'No-Anomaly' 'No-Anomaly' 'No-Anomaly' 'No-Anomaly' 'No-Anomaly'
 'No-Anomaly' 'No-Anomaly' 'Anomaly' 'Anomaly' 'Anomaly' 'No-Anomaly']


In [67]:

svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[2, 2],
       [1, 5]])